In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import pyvo as vo


## Checking column names for ref

In [4]:
from astroquery.gaia import Gaia
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')
print(gaiadr3_table)
for column in gaiadr3_table.columns:
  print(column.name)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152
solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pm

In [5]:
Gaia.ROW_LIMIT = -1  # Ensure the default row limit.
coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
j = Gaia.cone_search_async(coord, radius=u.Quantity(9.0, u.deg), columns=("source_id", "ra", "dec", "phot_g_mean_flux", "phot_g_mean_flux_error", "pm", "parallax", "parallax_error", "phot_bp_mean_flux", "phot_bp_mean_flux_error", "phot_rp_mean_flux", "phot_rp_mean_flux_error", "teff_gspphot", "teff_gspphot_lower", "teff_gspphot_upper", "logg_gspphot", "logg_gspphot_lower", "logg_gspphot_upper", "mh_gspphot", "mh_gspphot_upper", "mh_gspphot_lower", "bp_rp", "bp_g", "g_rp",    ))  
r = j.get_results()
r.pprint()  
r = r.to_pandas()

INFO: Query finished. [astroquery.utils.tap.core]
     source_id              ra         ...    g_rp             dist        
                           deg         ...    mag                          
------------------- ------------------ ... ---------- ---------------------
6636090334814214528  280.0002534562339 ... 0.53671074 0.0026043272506261527
6636090339112400000 279.99329161242713 ... 0.59051704 0.0033616678530916998
6636090339113063296 280.00510823916443 ...  0.9586029 0.0038498801828703495
6636090339112213760 279.99302733099194 ...   1.035902  0.004422603920589843
6636090334814217600 279.99469667562437 ...  0.5758667  0.004545515007418226
6636089583198816640  280.0061780746836 ...  0.5480366   0.00561391998241014
6636090334814218752 280.00127379417773 ... 0.73373413  0.005845777923125324
6636090334814213632  279.9913058372584 ... 0.88223267  0.006210490970134131
6636090339112308864 279.98843313892786 ...  0.7227535  0.007466944025525982
                ...                ...

In [11]:
r = r.drop('pseudocolour', axis=1)
r = r.drop('pseudocolour_error', axis=1)

# Remove rows with any NULL values
result = r.dropna()

## Getting Simbad otypes for these bad boys

In [16]:
# Initialize SIMBAD
simbad = Simbad()
simbad.ROW_LIMIT = -1
simbad.add_votable_fields( "plx_value", "V", "I", "J", "H", "K","G", "pmdec", "pmra", "hpx","ids", "otype")

# Define coordinates (random)
coordinates = SkyCoord(0, -90, unit=("deg", "deg"))
coordinates2 = SkyCoord(0, 90, unit=("deg", "deg"))

# Query region 1st half
result = simbad.query_region(coordinates, radius="90d0m",
                             criteria="otype = 'Ma*..' OR otype = 'MS*..' OR otype = 'Y*O..' OR otype = 'Ev*..' ")

filtered_result = result["main_id", "ra", "dec", "plx_value", "V", "I", "J", "H", "K","G", "pmdec", "pmra", 
                         "hpx", "ids", "otype"]
result_df = filtered_result.to_pandas() # Convert result to a Pandas DataFrame
filtered_result = result_df.dropna() # Remove rows with any NULL values
data1 = filtered_result.to_numpy() # Convert to numpy array


# Query region 2nd half
result2 = simbad.query_region(coordinates2, radius="90d0m",
                             criteria="otype = 'Ma*..' OR otype = 'MS*..' OR otype = 'Y*O..' OR otype = 'Ev*..' ")

filtered_result2 = result2["main_id", "ra", "dec", "plx_value", "V", "I", "J", "H", "K","G", "pmdec", "pmra",
                            "hpx", "ids", "otype"]
result_df2 = filtered_result2.to_pandas() # Convert result to a Pandas DataFrame
filtered_result2 = result_df2.dropna() # Remove rows with any NULL values
data2 = filtered_result2.to_numpy() # Convert to numpy array

# Combine the two arrays & save
data =np.row_stack((data1, data2))
#np.save("fullsky4cats", data)

## Combining both

## Other trashy stuff

In [34]:
from astroquery.vizier import Vizier

v = Vizier(columns=['_RAJ2000', '_DEJ2000','B-V', 'Vmag', 'Plx'],
           column_filters={"Vmag":">10"}, keywords=["optical", "xry"])  

gaia_catalog_list = Vizier.find_catalogs('GAIA DR3') 
print({k:v.description for k,v in gaia_catalog_list.items()})
coordinates = SkyCoord(135.9, -65.3, unit=("deg", "deg"))

result = Vizier(catalog='I/355/spectra').query_region(coordinates, radius="1d0m")

#result = result.to_pandas()


{'I/324': 'The Initial Gaia Source List (IGSL) (Smart, 2013)', 'I/337': 'Gaia DR1 (Gaia Collaboration, 2016)', 'I/345': 'Gaia DR2 (Gaia Collaboration, 2018)', 'I/347': 'Distances to 1.33 billion stars in Gaia DR2 (Bailer-Jones+, 2018)', 'I/350': 'Gaia EDR3 (Gaia Collaboration, 2020)', 'I/352': 'Distances to 1.47 billion stars in Gaia EDR3 (Bailer-Jones+, 2021)', 'I/355': 'Gaia DR3 Part 1. Main source (Gaia Collaboration, 2022)', 'I/356': 'Gaia DR3 Part 2. Extra-galactic (Gaia Collaboration, 2022)', 'I/357': 'Gaia DR3 Part 3. Non-single stars (Gaia Collaboration, 2022)', 'I/358': 'Gaia DR3 Part 4. Variability (Gaia Collaboration, 2022)', 'I/359': 'Gaia DR3 Part 5. Solar System (Gaia Collaboration, 2022)', 'I/360': 'Gaia DR3 Part 6. Performance verification (Gaia Collaboration, 2022)', 'I/361': 'Gaia Focused Product Release (Gaia FPR) (Gaia Collaboration, 2023)', 'IV/36': 'Gaia-IPHAS/KIS Value-Added Catalogues (Scaringi+, 2018)', 'VI/137': 'GaiaSimu Universe Model Snapshot (Robin+, 2012)

C:\Users\jcwin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\astroquery\vizier\core.py:908: UserWarning: xry : No such keyword
  warnings.warn("{val} : No such keyword".format(val=val))


AttributeError: 'TableList' object has no attribute 'to_pandas'

In [11]:
pdr = r.to_pandas()